In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
import os
from PIL import Image

from Config import Config
from Models import Tools, PreTrainedKPD

/home/wene/miniconda3/envs/mlenv/lib/python3.12/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [3]:
#Get csv files into dataframes
notes = pd.read_csv('CSVs/combined.csv')
preds = pd.read_csv('predictions_17december.csv')

#Get rid of extension name for some unmatch with jpg
preds['Image Name'] = preds['Image Name'].map(lambda s : os.path.splitext(s)[0])
notes['Image Name'] = notes['Image Name'].map(lambda s : os.path.splitext(s)[0])

#Get rid of duplicate annotation of the same image
notes = notes.groupby(['Image Name']).mean()

In [4]:
# match annotated coordinates with predicted ones using sql like inner join on column Image Name
df = preds.set_index('Image Name').join(notes, how='inner', lsuffix='_predicted', rsuffix='_real')

In [12]:
"""
it is finally not necessary to normalize here?
"""

#function to get images dimension for normalisation
#for combined and prediction are on the same original scale
def get_image_shape(file_path):
    with Image.open(file_path) as img:
        return img.size


shapes  = df.index.map(lambda s : get_image_shape(
    os.path.join(Config.images_folder_path, s + '.png')
    )
).to_frame( index=False,name=['X','Y']).values

df['X'] = shapes[:,0]
df['Y'] = shapes[:,1]

#Normalizing columns, also the heigt and width are note 
df = df.assign(
    x1_predicted=lambda d: d['x1_predicted'] / d['X'],
    y1_predicted=lambda d: d['y1_predicted'] / d['Y'],
    x2_predicted=lambda d: d['x2_predicted'] / d['X'],
    y2_predicted=lambda d: d['y2_predicted'] / d['Y'],
    x1_real=lambda d: d['x1_real'] / d['X'],
    y1_real=lambda d: d['y1_real'] / d['Y'],
    x2_real=lambda d: d['x2_real'] / d['X'],
    y2_real=lambda d: d['y2_real'] / d['Y'], 
).drop(['X','Y'], axis=1)

In [13]:
"""
Compute of distance error the same as the confidence model prediction was trained
but not on the real distance error of the KPD but on the distance error from annotations
"""
keypoints = torch.tensor(df[['x1_predicted','y1_predicted','x2_predicted','y2_predicted']].values)
outputs = torch.tensor(df[['x1_real','y1_real','x2_real','y2_real']].values)

clamp_threshold = 0.004

errors = (keypoints - outputs)**2
mse_per_vector = torch.mean(errors, dim=1, keepdim=True)
mse_per_vector_np = mse_per_vector.cpu().numpy()

# Step 1: Normalize by mean and variance
mean = mse_per_vector.mean()
std = mse_per_vector.std()
normalized_vector = (mse_per_vector - mean) / std

min_val = normalized_vector.min()
max_val = normalized_vector.max()

scaled_errors = (normalized_vector - min_val) / (max_val - min_val)

df['Error scale'] = scaled_errors

print(f"MSE loss {torch.nn.MSELoss()(outputs, keypoints)}")
print(f"Tools.py distance loss (normalized?) {Tools.get_distance_loss(outputs, keypoints)}")

MSE loss 0.045702337944739976
Tools.py distance loss (normalized?) 0.12729763552053083


In [14]:
print(f"The error mean is {df['Error scale'].mean()}")
print(f"The error variance is {df['Error scale'].var()}")
print(f"The error mean given the confidence is { (df['Error scale'] * df['Error estimate']).sum() / df['Error scale'].sum()}")

The error mean is 0.26820112739373847
The error variance is 0.010187071145561702
The error mean given the confidence is 0.2982082624144184


In [ ]:
#Confidence model